# Aufgabenstellung

- Erstelle eine kompakte Analyse zum aktuellen Film- und Serienangebot
- Was macht gute Filme aus?
- Was schauen die Leute gerne und warum?
- Auf der Basis deiner Analyse möchte Netflix in Zukunft selbst neue Filme und Serien produzieren.


## Bibliotheken und Datensätze importieren

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
movies = pd.read_csv("../input/imdb-extensive-dataset/IMDb movies.csv")
ratings = pd.read_csv("../input/imdb-extensive-dataset/IMDb ratings.csv")

In [ ]:
ratings = ratings.rename(columns={"imdb_title_id": "imdb_title_id2"})
data = movies.merge(ratings, left_on='imdb_title_id', right_on='imdb_title_id2')

## Datensatz analysieren

In [ ]:
data.head()

In [ ]:
data.tail()

In [ ]:
data.info()

In [ ]:
data.shape

## Daten bereinigen

### fehlende Werte als "missing" ersetzen

In [ ]:
data.director.fillna("missing", inplace=True)
data.writer.fillna("missing", inplace=True)
data.production_company.fillna("missing", inplace=True)
data.actors.fillna("missing", inplace=True) 
data.description.fillna("missing", inplace=True)
data.budget.fillna("missing", inplace=True)
data.worlwide_gross_income.fillna("missing", inplace=True)
data.country.fillna("missing", inplace=True)

In [ ]:
data["genre_main"] = data["genre"].apply(lambda x: x.split(", ")[0])
data["country_main"] = data["country"].apply(lambda x: x.split(", ")[0])
data=data.drop(["country", "title", "usa_gross_income", "metascore", "reviews_from_users", "reviews_from_critics", "votes_1", "votes_2", "votes_3", "votes_4", "votes_5", "votes_6", "votes_7", "votes_8","votes_9", "votes_10", "top1000_voters_rating", "top1000_voters_votes", "us_voters_rating", "us_voters_votes", "non_us_voters_rating", "non_us_voters_votes", "mean_vote", "median_vote", "weighted_average_vote", "genre" ], axis = 1)

In [ ]:
data.info()

## Visualisierung

- Welches Genre wird am meisten produziert?

In [ ]:
plt.figure(figsize=(20,10))
sns.countplot(y="genre_main", data=data, order = data['genre_main'].value_counts().index)

- Dauer der Filme in den verschiedenen Ländern

In [ ]:
data = data.replace({"year":"TV Movie 2019"}, "2019")
data["year"] = data["year"].astype(int)

In [ ]:
top = data["country_main"].value_counts()[:5].index.tolist()
TopMovies = data[data["country_main"].isin(top)]
TopMovies2 = TopMovies.groupby(["year", "country_main"]).mean()
TopMovies2 = TopMovies2.reset_index()
TopMovies2

In [ ]:
data[["duration","avg_vote"]].hist(figsize=(20,7))

In [ ]:
plt.figure(figsize=(20,10))
sns.lineplot(data=TopMovies2, x="year", y="duration",hue="country_main", markers=True, dashes=False)

- Genre gruppiert und Mittelwert erfasst

In [ ]:
Genres = data.groupby(["genre_main"]).mean()
Genres_Count = data["genre_main"].value_counts()
Genres = Genres.merge(Genres_Count, left_on=Genres.index, right_on=Genres_Count.index)
Genres["Average_Vote"] = Genres["total_votes"] / Genres["genre_main"]
Genres["Average_Votes_Male"] = Genres["males_allages_votes"] / Genres["genre_main"]
Genres["Average_Votes_Female"] = Genres["females_allages_votes"] / Genres["genre_main"]
Genres = Genres.rename(columns={"key_0": "Genre","genre_main":"Movie_count"})

- Was haben die Leute am meisten angesehen:

In [ ]:
sns.catplot(kind="bar",data=Genres,x="Movie_count", y="Genre")

- Wo haben die Leute im Durchschnitt am meisten gevoted?

In [ ]:
sns.catplot(kind="bar",data=Genres,x="Average_Vote", y="Genre")

In [ ]:
sns.catplot(kind="bar",data=Genres,x="Average_Votes_Male", y="Genre")

In [ ]:
sns.catplot(kind="bar",data=Genres,x="Average_Votes_Female", y="Genre")

In [ ]:
Genres[Genres["Genre"].isin(["Drama","Film-Noir"])][["Genre","Average_Vote","Average_Votes_Male","Average_Votes_Female","votes","Movie_count"]]